In [538]:
import csv
from time import sleep
from selenium.webdriver.common.keys import Keys
import selenium
from selenium.webdriver.common.by import By

def write_password(driver):
    sleep(1.5)
    password = driver.find_element(by=By.XPATH,
                                   value='//input[@autocomplete="current-password"]')
    password.send_keys('your password')
    password.send_keys(Keys.RETURN)

def get_tweet_data(post):
    head = post.find_element(By.XPATH, './/div[2]/div[2]/div[1]/div/div/div[1]/div[1]').text.split('\n')

    try:
        link = post.find_element(By.XPATH, './/div[2]/div[2]/div[2]/div[2]//a').get_attribute('href')
    except:
        link = ''

    try:
        time = post.find_element(By.XPATH, './/div[2]/div[2]/div[1]/div/div/div[1]/a/time').get_attribute('datetime')
    except:
        time = ''

    bottom = post.find_element(By.XPATH, './/div[2]/div[2]/div[2]')
    text = post.find_element(By.XPATH,'.//div[2]/div[2]/div[2]/div[1]').text
    nickname = head[1]
    name = head[0]
    reply = bottom.find_element(By.XPATH, './/div[@data-testid="reply"]').get_attribute('aria-label')

    try:
        retweet = bottom.find_element(By.XPATH, './/div[@data-testid="retweet"]').get_attribute('aria-label')
    except:
        retweet = bottom.find_element(By.XPATH, './/div[@data-testid="unretweet"]').get_attribute("aria-label")

    try:
        like = bottom.find_element(By.XPATH, './/div[@data-testid="unlike"]').get_attribute('aria-label')
    except:
        like = bottom.find_element(By.XPATH, './/div[@data-testid="like"]').get_attribute('aria-label')

    tweet = (nickname, name, text, link, reply, retweet, like)
    id = ''
    for i in tweet:
        id += i

    return [id, time, nickname, name, text, link, reply, retweet, like]


options = selenium.webdriver.ChromeOptions()

options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = selenium.webdriver.Chrome('drivers/chromedriver')

driver.get('https://twitter.com/login')
driver.maximize_window()
sleep(2)

/tmp/ipykernel_20181/2128851273.py:56: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = selenium.webdriver.Chrome('drivers/chromedriver')


In [539]:
username = driver.find_element(by=By.XPATH, value='//input[@autocomplete="username"]')
username.send_keys('your email')
username.send_keys(Keys.RETURN)

In [540]:
try:
    write_password(driver)
    sleep(2)
except:
    login = driver.find_element(by=By.XPATH,
                                value='//input[@autocomplete="on"]')
    login.send_keys('your username')
    login.send_keys(Keys.RETURN)
    sleep(1)

    write_password(driver)
    sleep(2)

In [541]:
driver.get('https://twitter.com/elonmusk')
sleep(4)

In [546]:
driver.refresh()
sleep(5)
data = []
tweets_ids = set()
# last_pos = driver.execute_script('return window.pageYOffset;')
# scrolling = True

body = driver.find_element(By.TAG_NAME, 'body')
prev_h = driver.execute_script("return window.pageYOffset;")

while True:
    if len(data) > 20:
        break

    for i in range(4):
        body.send_keys(Keys.PAGE_DOWN)

    sleep(2)

    page_cards = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
    for card in page_cards[-40:]:
        tweet = get_tweet_data(card)

        if tweet:
            tweet_id = tweet[0]

            if tweet_id not in tweets_ids:
                tweets_ids.add(tweet_id)
                tweet.pop(0)
                data.append(tweet)

    new_h = driver.execute_script("return window.pageYOffset;")

    if new_h == prev_h:
        for i in range(4):
            body.send_keys(Keys.PAGE_UP)
        sleep(1)

    else:
        prev_h = new_h



In [548]:
from operator import itemgetter

data.sort(key=itemgetter(0), reverse=True)

In [549]:
for i in data:
    print(i[0])


2022-05-01T06:23:15.000Z
2022-04-29T21:37:16.000Z
2022-04-29T21:19:50.000Z
2022-04-29T12:44:05.000Z
2022-04-29T12:28:10.000Z
2022-04-28T20:58:09.000Z
2022-04-28T20:53:49.000Z
2022-04-28T17:47:22.000Z
2022-04-28T02:14:58.000Z
2022-04-28T01:57:27.000Z
2022-04-28T01:57:27.000Z
2022-04-28T01:53:50.000Z
2022-04-28T01:53:50.000Z
2022-04-28T00:56:58.000Z
2022-04-28T00:13:47.000Z
2022-04-28T00:13:47.000Z
2022-04-27T23:41:36.000Z
2022-04-27T20:38:20.000Z
2022-04-27T20:38:20.000Z
2022-04-27T18:06:21.000Z
2022-04-27T17:28:36.000Z
2022-04-27T17:11:41.000Z
2022-04-27T08:07:56.000Z
2022-04-27T05:01:00.000Z
2022-04-27T04:55:45.000Z
2022-04-27T04:00:22.000Z
2022-04-26T20:15:32.000Z
2022-04-26T20:09:46.000Z


In [493]:
with open('musk_twitter.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['date', 'username', 'name', 'text', 'content', 'reply', 'retweet', 'like']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [537]:
len(data)

27